# Import library

In [20]:
# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

import os, sys, gc, warnings, random

import datetime
import dateutil.relativedelta

# Data manipulation
import pandas as pd 
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine learning
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, precision_recall_curve, roc_curve
from sklearn.model_selection import train_test_split, cross_val_score, KFold, StratifiedKFold, GroupKFold
from sklearn.ensemble import RandomForestClassifier

import lightgbm as lgb

from tqdm.notebook import trange, tqdm

from IPython.display import display

%matplotlib inline

pd.options.display.max_rows = 200
pd.options.display.max_columns = 200
pd.options.display.max_colwidth = 200

# Preparation

In [21]:
train_csv = pd.read_csv("/opt/ml/code/input/train.csv", parse_dates=['order_date'])

In [22]:
def seed_everything(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    

SEED = 42
seed_everything(SEED)

In [23]:
from datetime import date

def devide(x):
    answer = ''
    if x<0: answer = 'minus'
    elif x<20: answer = 'very low'
    elif x<40: answer = 'low'
    elif x<70: answer = 'middle-low'
    elif x<100: answer = 'middle'
    elif x<150: answer = 'middle-high'
    elif x<200: answer = 'almost'
    elif x<250: answer = 'high'
    elif x<270: answer = 'higgh'
    elif x<300: answer = 'hiigh'
    elif x<325: answer = 'veeery high'
    elif x<350: answer = 'very high'
    elif x<500: answer = 'very hiigh'
    elif x<1000: answer = 'super high'
    else: answer = 'epic high'
    return answer

def count_over(df,year_month):
    cid = 'customer_id'
    cust = df[df['year_month'] < year_month][cid].unique()
    count = pd.DataFrame({'customer_id':cust}).sort_values(cid)
    
    count['count'] = 0
    count = count.set_index(cid)
    while True:
        year_month = date(int(year_month.split('-')[0]),int(year_month.split('-')[1]),1) - dateutil.relativedelta.relativedelta(months=1)
        year_month = year_month.strftime('%Y-%m')
        total = df[df['year_month'] == year_month].groupby('customer_id').sum().reset_index()
        if total.empty: break
        cust = total[total['total']>300][cid].unique()
        for x in cust:
            count.at[x,'count']+=1
    return count.sort_values('customer_id').reset_index()['count']

def mean_during_usage(df, year_month):
    df = df.copy()
    cid = 'customer_id'
    cust = df[df['year_month'] < year_month][cid].unique()
    count = pd.DataFrame({'customer_id':cust}).sort_values(cid)
    
    count['count'] = 0
    count = count.set_index(cid)
    while True:
        year_month = date(int(year_month.split('-')[0]),int(year_month.split('-')[1]),1) - dateutil.relativedelta.relativedelta(months=1)
        year_month = year_month.strftime('%Y-%m')
        total = df[df['year_month'] == year_month].groupby('customer_id').sum().reset_index()
        if total.empty: break
        cust = total[total['total']>300][cid].unique()
        for x in cust:
            count.at[x,'count']+=1
    return count.sort_values('customer_id').reset_index()['count']
    


In [24]:
TOTAL_THRES = 300

'''
    입력인자로 받는 year_month에 대해 고객 ID별로 총 구매액이
    구매액 임계값을 넘는지 여부의 binary label을 생성하는 함수
'''
def generate_label(df, year_month, total_thres=TOTAL_THRES, print_log=False):
    df = df.copy()
    
    # year_month에 해당하는 label 데이터 생성
    df['year_month'] = df['order_date'].dt.strftime('%Y-%m')
    df.reset_index(drop=True, inplace=True)

    # year_month 이전 월의 고객 ID 추출
    cust = df[df['year_month']<year_month]['customer_id'].unique()
    # year_month에 해당하는 데이터 선택
    df = df[df['year_month']==year_month]
    
    # label 데이터프레임 생성
    label = pd.DataFrame({'customer_id':cust})
    label['year_month'] = year_month
    
    # year_month에 해당하는 고객 ID의 구매액의 합 계산
    grped = df.groupby(['customer_id','year_month'], as_index=False)[['total']].sum()
    
    # label 데이터프레임과 merge하고 구매액 임계값을 넘었는지 여부로 label 생성
    label = label.merge(grped, on=['customer_id','year_month'], how='left')
    label['total'].fillna(0.0, inplace=True)
    label['label'] = (label['total'] > total_thres).astype(int)

    # 고객 ID로 정렬
    label = label.sort_values('customer_id').reset_index(drop=True)
    if print_log: print(f'{year_month} - final label shape: {label.shape}')
    
    return label

In [47]:
from tqdm import tqdm

def feature_preprocessing(train, test, features, do_imputing=True):
    x_tr = train.copy()
    x_te = test.copy()
    
    # 범주형 피처 이름을 저장할 변수
    cate_cols = []

    # 레이블 인코딩
    for f in features:
        if x_tr[f].dtype.name == 'object': # 데이터 타입이 object(str)이면 레이블 인코딩
            cate_cols.append(f)
            le = LabelEncoder()
            # train + test 데이터를 합쳐서 레이블 인코딩 함수에 fit
            le.fit(list(x_tr[f].values) + list(x_te[f].values))
            
            # train 데이터 레이블 인코딩 변환 수행
            x_tr[f] = le.transform(list(x_tr[f].values))
            
            # test 데이터 레이블 인코딩 변환 수행
            x_te[f] = le.transform(list(x_te[f].values))

    print('categorical feature:', cate_cols)
    
    if do_imputing:
        impt = SimpleImputer(strategy='constant', fill_value=-2)
        x_tr[features] = impt.fit_transform(x_tr[features])
        x_te[features] = impt.transform(x_te[features])
    
    return x_tr, x_te

In [26]:
def make_feature(df, ref_date, period, feature, col_prefix):
    df = df.copy()
    ref_date = datetime.datetime.strptime(ref_date, "%Y-%m")
    if period[0]=='d':
        # period[1] > period[2]
        date_s = ref_date - dateutil.relativedelta.relativedelta(days=period[1])
        date_e = ref_date - dateutil.relativedelta.relativedelta(days=period[2])
    if period[0]=='m':
        # period[1] > period[2]
        date_s = ref_date - dateutil.relativedelta.relativedelta(months=period[1])
        date_e = ref_date - dateutil.relativedelta.relativedelta(months=period[2])
    date_s = date_s.strftime("%Y-%m-%d")
    date_e = date_e.strftime("%Y-%m-%d")
    
    df = df[(date_s<df['order_date'])&(date_e>df['order_date'])]

    # customer_id 기준으로 pandas group by 후 total, quantity, price 누적합 계산
    df['cumsum_total_by_cust_id'] = df.groupby(['customer_id'])['total'].cumsum()
    df['cumsum_quantity_by_cust_id'] = df.groupby(['customer_id'])['quantity'].cumsum()
    df['cumsum_price_by_cust_id'] = df.groupby(['customer_id'])['price'].cumsum()

    # product_id 기준으로 pandas group by 후 total, quantity, price 누적합 계산
    df['cumsum_total_by_prod_id'] = df.groupby(['product_id'])['total'].cumsum()
    df['cumsum_quantity_by_prod_id'] = df.groupby(['product_id'])['quantity'].cumsum()
    df['cumsum_price_by_prod_id'] = df.groupby(['product_id'])['price'].cumsum()
    
    # order_id 기준으로 pandas group by 후 total, quantity, price 누적합 계산
    df['cumsum_total_by_order_id'] = df.groupby(['order_id'])['total'].cumsum()
    df['cumsum_quantity_by_order_id'] = df.groupby(['order_id'])['quantity'].cumsum()
    df['cumsum_price_by_order_id'] = df.groupby(['order_id'])['price'].cumsum()

    # new baseline
    df['order_ts'] = df['order_date'].astype(np.int64)//1e9
    df['order_ts_diff'] = df.groupby(['customer_id'])['order_ts'].diff()
    df['quantity_diff'] = df.groupby(['customer_id'])['quantity'].diff()
    df['price_diff'] = df.groupby(['customer_id'])['price'].diff()
    df['total_diff'] = df.groupby(['customer_id'])['total'].diff()
    
    ret_data = pd.DataFrame()
    
    # group by aggretation 함수로 데이터 피처 생성
    df_agg = df.groupby(['customer_id']).agg(feature)

    # 멀티 레벨 컬럼을 사용하기 쉽게 1 레벨 컬럼명으로 변경
    new_cols = []
    cnt = 0
    for col in feature.keys():
        for stat in feature[col]:
            if type(stat) is str:
                new_cols.append(f'{col_prefix}-{col}-{stat}')
            else:
                new_cols.append(f'{col_prefix}-{col}-custom_{cnt}')
                cnt += 1
            

    df_agg.columns = new_cols
    df_agg.reset_index(inplace = True)

    df_agg['year_month'] = ref_date
    df_agg['year_month'] = df_agg['year_month'].dt.strftime('%Y-%m')

    ret_data = ret_data.append(df_agg)
    
    # ret_data = train_label.merge(all_train_data, on=['customer_id', 'year_month'], how='left')
    features = ret_data.drop(columns=['customer_id', 'year_month']).columns
    
    # ret_data 데이터 전처리 -> 수정 후 나중에 해도 될 듯?
    # x_tr, x_te = feature_preprocessing(all_train_data, test_data, features)
    
    print('ret_data.shape', ret_data.shape)
    
    return ret_data, features



In [33]:
def make_fixed_period_feature(df, ref_date, period, feature, col_prefix):
    df = df.copy()
    date_s = period[0]
    date_e = period[1]
    
    df = df[(date_s<df['order_date'])&(date_e>df['order_date'])]

    # customer_id 기준으로 pandas group by 후 total, quantity, price 누적합 계산
    df['cumsum_total_by_cust_id'] = df.groupby(['customer_id'])['total'].cumsum()
    df['cumsum_quantity_by_cust_id'] = df.groupby(['customer_id'])['quantity'].cumsum()
    df['cumsum_price_by_cust_id'] = df.groupby(['customer_id'])['price'].cumsum()

    # product_id 기준으로 pandas group by 후 total, quantity, price 누적합 계산
    df['cumsum_total_by_prod_id'] = df.groupby(['product_id'])['total'].cumsum()
    df['cumsum_quantity_by_prod_id'] = df.groupby(['product_id'])['quantity'].cumsum()
    df['cumsum_price_by_prod_id'] = df.groupby(['product_id'])['price'].cumsum()
    
    # order_id 기준으로 pandas group by 후 total, quantity, price 누적합 계산
    df['cumsum_total_by_order_id'] = df.groupby(['order_id'])['total'].cumsum()
    df['cumsum_quantity_by_order_id'] = df.groupby(['order_id'])['quantity'].cumsum()
    df['cumsum_price_by_order_id'] = df.groupby(['order_id'])['price'].cumsum()

    # new baseline
    df['order_ts'] = df['order_date'].astype(np.int64)//1e9
    df['order_ts_diff'] = df.groupby(['customer_id'])['order_ts'].diff()
    df['quantity_diff'] = df.groupby(['customer_id'])['quantity'].diff()
    df['price_diff'] = df.groupby(['customer_id'])['price'].diff()
    df['total_diff'] = df.groupby(['customer_id'])['total'].diff()
    
    ret_data = pd.DataFrame()
    
    # group by aggretation 함수로 데이터 피처 생성
    df_agg = df.groupby(['customer_id']).agg(feature)

    # 멀티 레벨 컬럼을 사용하기 쉽게 1 레벨 컬럼명으로 변경
    new_cols = []
    cnt = 0
    for col in feature.keys():
        for stat in feature[col]:
            if type(stat) is str:
                new_cols.append(f'{col_prefix}-{col}-{stat}')
            else:
                new_cols.append(f'{col_prefix}-{col}-custom_{cnt}')
                cnt += 1
            

    df_agg.columns = new_cols
    df_agg.reset_index(inplace = True)

    df_agg['year_month'] = datetime.datetime.strptime(ref_date, "%Y-%m")
    df_agg['year_month'] = df_agg['year_month'].dt.strftime('%Y-%m')

    ret_data = ret_data.append(df_agg)
    
    # ret_data = train_label.merge(all_train_data, on=['customer_id', 'year_month'], how='left')
    features = ret_data.drop(columns=['customer_id', 'year_month']).columns
    
    # ret_data 데이터 전처리 -> 수정 후 나중에 해도 될 듯?
    # x_tr, x_te = feature_preprocessing(all_train_data, test_data, features)
    
    print('ret_data.shape', ret_data.shape)
    
    return ret_data, features



In [28]:
def range_func(x):
    max_val = np.max(x)
    min_val = np.min(x)
    range_val = max_val - min_val
    return range_val


def iqr_func(x):
    q3, q1 = np.percentile(x, [75, 25])
    iqr = q3 - q1
    return iqr

In [10]:
def feature_engineering3(df, test_ref_date, train_ref_date, period_rule, feature_rule, total_thres):
    df = df.copy()
    
    # 1. customer_id 추출
    train_label = generate_label(df, train_ref_date, total_thres)[['customer_id','year_month','label']]
    test_label = generate_label(df, test_ref_date, total_thres)[['customer_id','year_month','label']]
    
    # 2. period_rule과 feature_rule에 따라서 위 customer_id에 대응하는 값을 train_ref_date에서 추출
    # 3. 2에서 추출한 dataframe을 병합하여 train_data
    train_data = train_label.copy()
    # train_df_list = []
    train_feat_list = None
    if len(period_rule)!=len(feature_rule):
        raise("Error:\n\tlen(period_rule)!=len(feature_rule)")
    for i in range(len(period_rule)):
        res_tuple = make_feature(df, train_ref_date, period_rule[i], feature_rule[i], i)
        train_data = train_data.merge(res_tuple[0], on=['customer_id', 'year_month'], how='left')
        if train_feat_list is None:
            train_feat_list = res_tuple[1].copy()
        else:
            train_feat_list = train_feat_list.append(res_tuple[1].copy())
    
    df['year_month'] = [str(x)[:7] for x in df['order_date'].copy()[:]]
    train = df[(df['year_month']<'2011-11')&(df['year_month']>'2011-01')][:]
    train_month_count = [len(set(train[train['customer_id']==cust]['year_month'])) for cust in train_data['customer_id'].copy()]
    train_month_sum = [train[train['customer_id']==cust]['total'].sum() for cust in train_data['customer_id'].copy()]
    train_month_mean = [y/x if x>0 else 0 for x, y in zip(train_month_count, train_month_sum)]
    train_mean_cat = [devide(x) for x in train_month_mean]
    train_data['count_m'] = train_month_count
    train_data['sum_m'] = train_month_sum
    train_data['mean_m'] = train_month_mean
    train_data['mean_cat'] = train_mean_cat
    train_feat_list = train_feat_list.append(pd.Index(['count_m', 'sum_m', 'mean_m', 'mean_cat']))

    
    # 4. period_rule과 feature_rule에 따라서 위 customer_id에 대응하는 값을 test_ref_date에서 추출
    # 5. 4에서 추출한 dataframe을 병합하여 test_data
    test_data = test_label.copy()
    # test_df_list = []
    test_feat_list = None
    for i in range(len(period_rule)):
        res_tuple = make_feature(df, test_ref_date, period_rule[i], feature_rule[i], i)
        test_data = test_data.merge(res_tuple[0], on=['customer_id', 'year_month'], how='left')
        if test_feat_list is None:
            test_feat_list = res_tuple[1].copy()
        else:
            test_feat_list = test_feat_list.append(res_tuple[1].copy())
    
    test = df[(df['year_month']<'2011-12')&(df['year_month']>'2011-02')][:]
    test_month_count = [len(set(test[test['customer_id']==cust]['year_month'])) for cust in test_data['customer_id'].copy()]
    test_month_sum = [test[test['customer_id']==cust]['total'].sum() for cust in test_data['customer_id'].copy()]
    test_month_mean = [y/x if x>0 else 0 for x, y in zip(test_month_count, test_month_sum)]
    test_mean_cat = [devide(x) for x in test_month_mean]
    test_data['count_m'] = test_month_count
    test_data['sum_m'] = test_month_sum
    test_data['mean_m'] = test_month_mean
    test_data['mean_cat'] = test_mean_cat
    test_feat_list = test_feat_list.append(pd.Index(['count_m', 'sum_m', 'mean_m', 'mean_cat']))
    
    # train_feat_list = train_data.drop(columns=['customer_id', 'label', 'year_month']).columns # 한번 해봄
    
    # 6. train_data, train_label, test_data, test_label, features 최종 반환
    if (train_feat_list!=test_feat_list).sum()!=0: # len(train_feat_list)!=len(test_feat_list)
        raise("Error:\n\t(train_feat_list!=test_feat_list).sum()!=0")
    x_tr, x_te = feature_preprocessing(train_data, test_data, train_feat_list, do_imputing=True)
    
    print('x_tr.shape', x_tr.shape, ', x_te.shape', x_te.shape)
    
    return x_tr, x_te, train_label['label'], train_feat_list


In [29]:
def do_anything(df, features):
    df = df.copy()
    for f in features:
        if not pd.api.types.is_numeric_dtype(df[f]):
            continue
        if f=='mean_cat':
            continue
        new_colname = f+"_squared_n_log"
        df[new_colname] = [x*2 for x in df[f]]
        df[new_colname] = [np.log(x+0.5) for x in df[new_colname]]
    return df

In [46]:
agg_basic = ['mean','max','min','sum','count','std','skew']
agg_custom = ['mean','max','min','sum','count','std','skew', 'median', range_func, iqr_func]
agg_custom_2 = ['mean','max','min','sum','count','std','skew', 'median', range_func]
agg_dict = {
    'quantity': agg_custom,
    'price': agg_custom,
    'total': agg_custom,
    'cumsum_total_by_cust_id': agg_custom,
    'cumsum_quantity_by_cust_id': agg_custom,
    'cumsum_price_by_cust_id': agg_custom,
    'cumsum_total_by_prod_id': agg_custom,
    'cumsum_quantity_by_prod_id': agg_custom,
    'cumsum_price_by_prod_id': agg_custom,
    'cumsum_total_by_order_id': agg_custom,
    'cumsum_quantity_by_order_id': agg_custom,
    'cumsum_price_by_order_id': agg_custom,
    'order_id': ['nunique'],
    'product_id': ['nunique'],
    "order_ts":["first", "last"],
    "order_ts_diff":agg_custom_2,
    "quantity_diff":agg_custom_2,
    "price_diff":agg_custom_2,
    "total_diff":agg_custom_2
}


def feature_engineering4(df, test_ref_date, train_ref_date, period_rule, feature_rule, total_thres):
    df = df.copy()
    
    # 1. customer_id 추출
    train_label = generate_label(df, train_ref_date, total_thres)[['customer_id','year_month','label']]
    test_label = generate_label(df, test_ref_date, total_thres)[['customer_id','year_month','label']]
    
    # 2. period_rule과 feature_rule에 따라서 위 customer_id에 대응하는 값을 train_ref_date에서 추출
    # 3. 2에서 추출한 dataframe을 병합하여 train_data
    train_data = train_label.copy()
    # train_df_list = []
    train_feat_list = None
    if len(period_rule)!=len(feature_rule):
        raise("Error:\n\tlen(period_rule)!=len(feature_rule)")
    for i in range(len(period_rule)):
        res_tuple = make_feature(df, train_ref_date, period_rule[i], feature_rule[i], i)
        train_data = train_data.merge(res_tuple[0], on=['customer_id', 'year_month'], how='left')
        if train_feat_list is None:
            train_feat_list = res_tuple[1].copy()
        else:
            train_feat_list = train_feat_list.append(res_tuple[1].copy())
    #res_tuple = make_fixed_period_feature(df, train_ref_date, ("2010-11", "2011-11"), agg_dict, len(period_rule))
    #train_data = train_data.merge(res_tuple[0], on=['customer_id', 'year_month'], how='left')
    #train_feat_list = train_feat_list.append(res_tuple[1].copy())
    
    
    df['year_month'] = [str(x)[:7] for x in df['order_date'].copy()[:]]
    train = df[(df['year_month']<'2011-11')&(df['year_month']>'2010-12')][:]
    train_month_count = [len(set(train[train['customer_id']==cust]['year_month'])) for cust in train_data['customer_id'].copy()]
    train_month_sum = [train[train['customer_id']==cust]['total'].sum() for cust in train_data['customer_id'].copy()]
    train_month_mean = [y/x if x>0 else 0 for x, y in zip(train_month_count, train_month_sum)]
    train_mean_cat = [devide(x) for x in train_month_mean]
    train_data['count_m'] = train_month_count
    train_data['sum_m'] = train_month_sum
    train_data['mean_m'] = train_month_mean
    train_data['mean_cat'] = train_mean_cat
    train_feat_list = train_feat_list.append(pd.Index(['count_m', 'sum_m', 'mean_m', 'mean_cat']))

    
    # 4. period_rule과 feature_rule에 따라서 위 customer_id에 대응하는 값을 test_ref_date에서 추출
    # 5. 4에서 추출한 dataframe을 병합하여 test_data
    test_data = test_label.copy()
    # test_df_list = []
    test_feat_list = None
    for i in range(len(period_rule)):
        res_tuple = make_feature(df, test_ref_date, period_rule[i], feature_rule[i], i)
        test_data = test_data.merge(res_tuple[0], on=['customer_id', 'year_month'], how='left')
        if test_feat_list is None:
            test_feat_list = res_tuple[1].copy()
        else:
            test_feat_list = test_feat_list.append(res_tuple[1].copy())
    #res_tuple = make_fixed_period_feature(df, test_ref_date, ("2010-11", "2011-11"), agg_dict, len(period_rule))
    #test_data = test_data.merge(res_tuple[0], on=['customer_id', 'year_month'], how='left')
    #test_feat_list = test_feat_list.append(res_tuple[1].copy())
    
    test = df[(df['year_month']<'2011-12')&(df['year_month']>'2011-01')][:]
    test_month_count = [len(set(test[test['customer_id']==cust]['year_month'])) for cust in test_data['customer_id'].copy()]
    test_month_sum = [test[test['customer_id']==cust]['total'].sum() for cust in test_data['customer_id'].copy()]
    test_month_mean = [y/x if x>0 else 0 for x, y in zip(test_month_count, test_month_sum)]
    test_mean_cat = [devide(x) for x in test_month_mean]
    test_data['count_m'] = test_month_count
    test_data['sum_m'] = test_month_sum
    test_data['mean_m'] = test_month_mean
    test_data['mean_cat'] = test_mean_cat
    test_feat_list = test_feat_list.append(pd.Index(['count_m', 'sum_m', 'mean_m', 'mean_cat']))
    
    train_feat_list = train_data.drop(columns=['customer_id', 'label', 'year_month']).columns # 한번 해봄
    test_feat_list = test_data.drop(columns=['customer_id', 'label', 'year_month']).columns
    
    # 6. train_data, train_label, test_data, test_label, features 최종 반환
    if (train_feat_list!=test_feat_list).sum()!=0: # len(train_feat_list)!=len(test_feat_list)
        raise("Error:\n\t(train_feat_list!=test_feat_list).sum()!=0")
    x_tr, x_te = feature_preprocessing(train_data, test_data, train_feat_list, do_imputing=True)
    
    # x_tr = do_anything(x_tr, train_feat_list)
    # x_te = do_anything(x_te, train_feat_list)
    # train_feat_list = x_tr.drop(columns=['customer_id', 'label', 'year_month']).columns
    
    print('x_tr.shape', x_tr.shape, ', x_te.shape', x_te.shape)
    
    return x_tr, x_te, train_label['label'], train_feat_list


In [31]:
def make_lgb_oof_prediction(train, y, test, features, categorical_features='auto', model_params=None, folds=10):
    x_train = train[features]
    x_test = test[features]
    
    # 테스트 데이터 예측값을 저장할 변수
    test_preds = np.zeros(x_test.shape[0])
    
    # Out Of Fold Validation 예측 데이터를 저장할 변수
    y_oof = np.zeros(x_train.shape[0])
    
    # 폴드별 평균 Validation 스코어를 저장할 변수
    score = 0
    
    # 피처 중요도를 저장할 데이터 프레임 선언
    fi = pd.DataFrame()
    fi['feature'] = features
    
    # Stratified K Fold 선언
    skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=SEED)

    for fold, (tr_idx, val_idx) in enumerate(skf.split(x_train, y)):
        # train index, validation index로 train 데이터를 나눔
        x_tr, x_val = x_train.loc[tr_idx, features], x_train.loc[val_idx, features]
        y_tr, y_val = y[tr_idx], y[val_idx]
        
        print(f'fold: {fold+1}, x_tr.shape: {x_tr.shape}, x_val.shape: {x_val.shape}')

        # LightGBM 데이터셋 선언
        dtrain = lgb.Dataset(x_tr, label=y_tr)
        dvalid = lgb.Dataset(x_val, label=y_val)
        
        # LightGBM 모델 훈련
        clf = lgb.train(
            model_params,
            dtrain,
            valid_sets=[dtrain, dvalid], # Validation 성능을 측정할 수 있도록 설정
            categorical_feature=categorical_features,
            verbose_eval=200
        )

        # Validation 데이터 예측
        val_preds = clf.predict(x_val)
        
        # Validation index에 예측값 저장 
        y_oof[val_idx] = val_preds
        
        # 폴드별 Validation 스코어 측정
        print(f"Fold {fold + 1} | AUC: {roc_auc_score(y_val, val_preds)}")
        print('-'*80)

        # score 변수에 폴드별 평균 Validation 스코어 저장
        score += roc_auc_score(y_val, val_preds) / folds
        
        # 테스트 데이터 예측하고 평균해서 저장
        test_preds += clf.predict(x_test) / folds
        
        # 폴드별 피처 중요도 저장
        fi[f'fold_{fold+1}'] = clf.feature_importance()

        del x_tr, x_val, y_tr, y_val
        gc.collect()
        
    print(f"\nMean AUC = {score}") # 폴드별 Validation 스코어 출력
    print(f"OOF AUC = {roc_auc_score(y, y_oof)}") # Out Of Fold Validation 스코어 출력
        
    # 폴드별 피처 중요도 평균값 계산해서 저장 
    fi_cols = [col for col in fi.columns if 'fold_' in col]
    fi['importance'] = fi[fi_cols].mean(axis=1)
    
    return y_oof, test_preds, fi


# Train

In [12]:
agg_basic = ['mean','max','min','sum','count','std','skew']
agg_custom = ['mean','max','min','sum','count','std','skew', 'median', range_func, iqr_func]
agg_custom_2 = ['mean','max','min','sum','count','std','skew', 'median', range_func]
agg_dict = {
    'quantity': agg_custom,
    'price': agg_custom,
    'total': agg_custom,
    'cumsum_total_by_cust_id': agg_custom,
    'cumsum_quantity_by_cust_id': agg_custom,
    'cumsum_price_by_cust_id': agg_custom,
    'cumsum_total_by_prod_id': agg_custom,
    'cumsum_quantity_by_prod_id': agg_custom,
    'cumsum_price_by_prod_id': agg_custom,
    'cumsum_total_by_order_id': agg_custom,
    'cumsum_quantity_by_order_id': agg_custom,
    'cumsum_price_by_order_id': agg_custom,
    'order_id': ['nunique'],
    'product_id': ['nunique'],
    "order_ts":["first", "last"],
    "order_ts_diff":agg_custom_2,
    "quantity_diff":agg_custom_2,
    "price_diff":agg_custom_2,
    "total_diff":agg_custom_2
}


## trial 1

In [48]:
model = 'baseline3'
print('baseline model:', model)


TOTAL_THRES = 300 # 구매액 임계값
SEED = 42 # 랜덤 시드
seed_everything(SEED) # 시드 고정


data_dir = './input'
model_dir = './model'
output_dir = './output'


# 데이터 파일 읽기
data = pd.read_csv(data_dir + '/train.csv', parse_dates=['order_date'])

# 예측할 연월 설정
year_month = '2011-12'


period_rule = [("d", 7, 0), ("d", 14, 7), ("d", 30, 14), ("m", 2, 1), ("m", 3, 2), ("m", 6, 3), ("m", 6, 0), ("m", 9, 6), ("m", 12, 9), ("m", 15, 12), ('m', 22, 0)]
feature_rule = [
    agg_dict for _ in range(len(period_rule))
]


# 피처 엔지니어링 실행
train, test, y, features = feature_engineering4(data, year_month, '2011-11', period_rule, feature_rule, 300)


print(f"train type: {type(train)}")
print(f"test type: {type(test)}")
print(f"y type: {type(y)}")
print(f"features type: {type(features)}")

baseline model: baseline3
ret_data.shape (455, 162)
ret_data.shape (408, 162)
ret_data.shape (858, 162)
ret_data.shape (1302, 162)
ret_data.shape (980, 162)
ret_data.shape (2064, 162)
ret_data.shape (3234, 162)
ret_data.shape (1906, 162)
ret_data.shape (2281, 162)
ret_data.shape (2560, 162)
ret_data.shape (5620, 162)
ret_data.shape (512, 162)
ret_data.shape (614, 162)
ret_data.shape (1112, 162)
ret_data.shape (1425, 162)
ret_data.shape (1302, 162)
ret_data.shape (1993, 162)
ret_data.shape (3499, 162)
ret_data.shape (2027, 162)
ret_data.shape (1749, 162)
ret_data.shape (2936, 162)
ret_data.shape (5750, 162)
categorical feature: ['mean_cat']
x_tr.shape (5722, 1767) , x_te.shape (5914, 1767)
train type: <class 'pandas.core.frame.DataFrame'>
test type: <class 'pandas.core.frame.DataFrame'>
y type: <class 'pandas.core.series.Series'>
features type: <class 'pandas.core.indexes.base.Index'>


In [49]:
model_params = {
    'objective': 'binary', # 이진 분류
    'boosting_type': 'gbdt',
    'metric': 'auc', # 평가 지표 설정
    'feature_fraction': 0.7, # 피처 샘플링 비율
    'bagging_fraction': 0.6, # 데이터 샘플링 비율
    'bagging_freq': 1,
    'n_estimators': 10000, # 트리 개수
    'early_stopping_rounds': 300,
    'seed': SEED,
    'verbose': -1,
    'n_jobs': -1,    
}


# Cross Validation Out Of Fold로 LightGBM 모델 훈련 및 예측
y_oof, test_preds, fi = make_lgb_oof_prediction(train, y, test, features, model_params=model_params, folds=10)

# 테스트 결과 제출 파일 읽기
sub = pd.read_csv('./input/sample_submission.csv')

# 테스트 예측 결과 저장
sub['probability'] = test_preds


os.makedirs(output_dir, exist_ok=True)
# 제출 파일 쓰기
sub.to_csv(os.path.join('output_baseline3_final_1.csv'), index=False)
# 기존 모델

fold: 1, x_tr.shape: (5149, 1764), x_val.shape: (573, 1764)
Training until validation scores don't improve for 300 rounds
[200]	training's auc: 0.999636	valid_1's auc: 0.803687
Early stopping, best iteration is:
[33]	training's auc: 0.968537	valid_1's auc: 0.811572
Fold 1 | AUC: 0.811572205974901
--------------------------------------------------------------------------------
fold: 2, x_tr.shape: (5149, 1764), x_val.shape: (573, 1764)
Training until validation scores don't improve for 300 rounds
[200]	training's auc: 0.999598	valid_1's auc: 0.79334
Early stopping, best iteration is:
[5]	training's auc: 0.885821	valid_1's auc: 0.823233
Fold 2 | AUC: 0.8232332580609337
--------------------------------------------------------------------------------
fold: 3, x_tr.shape: (5150, 1764), x_val.shape: (572, 1764)
Training until validation scores don't improve for 300 rounds
[200]	training's auc: 0.99961	valid_1's auc: 0.808482
Early stopping, best iteration is:
[16]	training's auc: 0.926319	va

## trial 2

In [52]:
model = 'baseline3'
print('baseline model:', model)


TOTAL_THRES = 300 # 구매액 임계값
SEED = 42 # 랜덤 시드
seed_everything(SEED) # 시드 고정


data_dir = './input' 
model_dir = './model'
output_dir = './output'


# 데이터 파일 읽기
data = pd.read_csv(data_dir + '/train.csv', parse_dates=['order_date'])

# 예측할 연월 설정
year_month = '2011-12'


period_rule = [("m", 1, 0), ("m", 2, 1), ("m", 3, 2), ("m", 6, 3), ("m", 9, 6), ("m", 12, 9), ("m", 15, 12), ("m", 18, 15), ('m', 22, 0)]
feature_rule = [
    agg_dict for _ in range(len(period_rule))
]


# 피처 엔지니어링 실행
train, test, y, features = feature_engineering4(data, year_month, '2011-11', period_rule, feature_rule, 300)


print(f"train type: {type(train)}")
print(f"test type: {type(test)}")
print(f"y type: {type(y)}")
print(f"features type: {type(features)}")

baseline model: baseline3
ret_data.shape (1425, 162)
ret_data.shape (1302, 162)
ret_data.shape (980, 162)
ret_data.shape (2064, 162)
ret_data.shape (1906, 162)
ret_data.shape (2281, 162)
ret_data.shape (2560, 162)
ret_data.shape (2086, 162)
ret_data.shape (5620, 162)
ret_data.shape (1711, 162)
ret_data.shape (1425, 162)
ret_data.shape (1302, 162)
ret_data.shape (1993, 162)
ret_data.shape (2027, 162)
ret_data.shape (1749, 162)
ret_data.shape (2936, 162)
ret_data.shape (2008, 162)
ret_data.shape (5750, 162)
categorical feature: ['mean_cat']
x_tr.shape (5722, 1447) , x_te.shape (5914, 1447)
train type: <class 'pandas.core.frame.DataFrame'>
test type: <class 'pandas.core.frame.DataFrame'>
y type: <class 'pandas.core.series.Series'>
features type: <class 'pandas.core.indexes.base.Index'>


In [53]:
model_params = {
    'objective': 'binary', # 이진 분류
    'boosting_type': 'gbdt',
    'metric': 'auc', # 평가 지표 설정
    'feature_fraction': 0.7, # 피처 샘플링 비율
    'bagging_fraction': 0.6, # 데이터 샘플링 비율
    'bagging_freq': 1,
    'n_estimators': 10000, # 트리 개수
    'early_stopping_rounds': 300,
    'seed': SEED,
    'verbose': -1,
    'n_jobs': -1,    
}


# Cross Validation Out Of Fold로 LightGBM 모델 훈련 및 예측
y_oof, test_preds, fi = make_lgb_oof_prediction(train, y, test, features, model_params=model_params, folds=10)

# 테스트 결과 제출 파일 읽기
sub = pd.read_csv('./input/sample_submission.csv')

# 테스트 예측 결과 저장
sub['probability'] = test_preds


os.makedirs(output_dir, exist_ok=True)
# 제출 파일 쓰기
sub.to_csv(os.path.join('output_baseline3_final_4.csv'), index=False)
# 1달 단위

fold: 1, x_tr.shape: (5149, 1444), x_val.shape: (573, 1444)
Training until validation scores don't improve for 300 rounds
[200]	training's auc: 0.999633	valid_1's auc: 0.802354
Early stopping, best iteration is:
[7]	training's auc: 0.898671	valid_1's auc: 0.822659
Fold 1 | AUC: 0.8226594602598748
--------------------------------------------------------------------------------
fold: 2, x_tr.shape: (5149, 1444), x_val.shape: (573, 1444)
Training until validation scores don't improve for 300 rounds
[200]	training's auc: 0.99958	valid_1's auc: 0.797301
Early stopping, best iteration is:
[24]	training's auc: 0.948124	valid_1's auc: 0.807963
Fold 2 | AUC: 0.8079628327101765
--------------------------------------------------------------------------------
fold: 3, x_tr.shape: (5150, 1444), x_val.shape: (572, 1444)
Training until validation scores don't improve for 300 rounds
[200]	training's auc: 0.999587	valid_1's auc: 0.823565
Early stopping, best iteration is:
[16]	training's auc: 0.926938	

## trial 3

In [54]:
model = 'baseline3'
print('baseline model:', model)


TOTAL_THRES = 300 # 구매액 임계값
SEED = 42 # 랜덤 시드
seed_everything(SEED) # 시드 고정


data_dir = './input' 
model_dir = './model'
output_dir = './output'


# 데이터 파일 읽기
data = pd.read_csv(data_dir + '/train.csv', parse_dates=['order_date'])

# 예측할 연월 설정
year_month = '2011-12'


period_rule = [("m", 2, 0), ("m", 4, 2), ("m", 6, 4), ("m", 8, 6), ("m", 10, 8), ("m", 12, 10), ("m", 15, 12), ("m", 18, 15), ('m', 22, 0)]
feature_rule = [
    agg_dict for _ in range(len(period_rule))
]


# 피처 엔지니어링 실행
train, test, y, features = feature_engineering4(data, year_month, '2011-11', period_rule, feature_rule, 300)


print(f"train type: {type(train)}")
print(f"test type: {type(test)}")
print(f"y type: {type(y)}")
print(f"features type: {type(features)}")


baseline model: baseline3
ret_data.shape (2175, 162)
ret_data.shape (1540, 162)
ret_data.shape (1666, 162)
ret_data.shape (1573, 162)
ret_data.shape (1282, 162)
ret_data.shape (2033, 162)
ret_data.shape (2560, 162)
ret_data.shape (2086, 162)
ret_data.shape (5620, 162)
ret_data.shape (2446, 162)
ret_data.shape (1817, 162)
ret_data.shape (1629, 162)
ret_data.shape (1579, 162)
ret_data.shape (1473, 162)
ret_data.shape (1369, 162)
ret_data.shape (2936, 162)
ret_data.shape (2008, 162)
ret_data.shape (5750, 162)
categorical feature: ['mean_cat']
x_tr.shape (5722, 1447) , x_te.shape (5914, 1447)
train type: <class 'pandas.core.frame.DataFrame'>
test type: <class 'pandas.core.frame.DataFrame'>
y type: <class 'pandas.core.series.Series'>
features type: <class 'pandas.core.indexes.base.Index'>


In [56]:
model_params = {
    'objective': 'binary', # 이진 분류
    'boosting_type': 'gbdt',
    'metric': 'auc', # 평가 지표 설정
    'feature_fraction': 0.7, # 피처 샘플링 비율
    'bagging_fraction': 0.6, # 데이터 샘플링 비율
    'bagging_freq': 1,
    'n_estimators': 10000, # 트리 개수
    'early_stopping_rounds': 300,
    'seed': SEED,
    'verbose': -1,
    'n_jobs': -1,    
}


# Cross Validation Out Of Fold로 LightGBM 모델 훈련 및 예측
y_oof, test_preds, fi = make_lgb_oof_prediction(train, y, test, features, model_params=model_params, folds=10)

# 테스트 결과 제출 파일 읽기
sub = pd.read_csv('./input/sample_submission.csv')

# 테스트 예측 결과 저장
sub['probability'] = test_preds


os.makedirs(output_dir, exist_ok=True)
# 제출 파일 쓰기
sub.to_csv(os.path.join('output_baseline3_final_5.csv'), index=False)
# 2달 단위

fold: 1, x_tr.shape: (5149, 1444), x_val.shape: (573, 1444)
Training until validation scores don't improve for 300 rounds
[200]	training's auc: 0.999645	valid_1's auc: 0.808333
Early stopping, best iteration is:
[24]	training's auc: 0.947007	valid_1's auc: 0.82141
Fold 1 | AUC: 0.8214100618220856
--------------------------------------------------------------------------------
fold: 2, x_tr.shape: (5149, 1444), x_val.shape: (573, 1444)
Training until validation scores don't improve for 300 rounds
[200]	training's auc: 0.999564	valid_1's auc: 0.804557
Early stopping, best iteration is:
[22]	training's auc: 0.946067	valid_1's auc: 0.813034
Fold 2 | AUC: 0.8130344648872765
--------------------------------------------------------------------------------
fold: 3, x_tr.shape: (5150, 1444), x_val.shape: (572, 1444)
Training until validation scores don't improve for 300 rounds
[200]	training's auc: 0.999588	valid_1's auc: 0.823602
Early stopping, best iteration is:
[7]	training's auc: 0.894222	